#### 토큰화

In [ ]:
text="Tokenizing text is a core task of NLP"
tokenized_text=list(text)
print(tokenized_text)

In [ ]:
# 각 토큰 수치화
token2idx={ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)

In [ ]:
# 토큰화된 텍스트를 정수 리스트로 변환
input_ids=[token2idx[token] for token in tokenized_text]
print(input_ids)

In [ ]:
import torch
import torch.nn.functional as F

input_ids=torch.tensor(input_ids)
one_hot_encodings=F.one_hot(input_ids,num_classes=len(token2idx))
one_hot_encodings.shape

In [ ]:
print(f"토큰: {tokenized_text[0]}")
print(f"텐서 인덱스: {input_ids[0]}")
print(f"원-핫 인코딩: {one_hot_encodings[0]}")

#### 단어 토큰화

In [ ]:
tokenized_text=text.split()
print(tokenized_text)

In [ ]:
from transformers import AutoTokenizer

model_ckpt="distilbert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
from transformers import DistilBertTokenizer

distilbert_tokenizer=DistilBertTokenizer.from_pretrained(model_ckpt)

In [ ]:
encoded_text=tokenizer(text)
print(encoded_text)

In [ ]:
# 토크나이저의 convert_ids_to_convert() 메서드 사용해서 다시 토큰으로 변환
tokens=tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

In [ ]:
print(tokenizer.convert_tokens_to_string(tokens))

In [ ]:
tokenizer.vocab_size

In [ ]:
tokenizer.model_max_length

In [ ]:
tokenizer.model_input_names

#### 트랜스포머를 특성 추출기로 사용하기

In [ ]:
# 사전 훈련된 모델 사용
from transformers import AutoModel
model_ckpt="distilbert-base-uncased"
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=AutoModel.from_pretrained(model_ckpt).to(device)

In [ ]:
# 마지막 은닉 상태 추출하기
text="this is a test"
inputs=tokenizer(text,return_tensors="pt") # 문자열 인코딩
print(f"입력 텐서 크기: {inputs['input_ids'].size()}")

In [ ]:
inputs={k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
  outputs=model(**inputs)
print(outputs)

In [ ]:
# last hidden state 크기 확인
outputs.last_hidden_state.size() # batch_size x n_tokens x hidden_dim

In [ ]:
outputs.last_hidden_state[:,0].size()

In [ ]:
def extract_hidden_states(batch):
  # 모델 입력을 GPU로 옮깁니다.
  inputs={k:v.to(device) for k,v in batch.items()
          if k in tokenizer.model_input_names}
  # 마지막 은닉 상태를 추출
  with torch.no_grad():
    last_hidden_state=model(**inputs).last_hidden_state
  # [CLS] 토큰에 대한 벡터를 반환
  return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}